In [8]:
import os
import sys
import urllib.request
import datetime
import time
import json

client_id = "M4g9rW2aSU0zNVXbp0Ve"
client_secret = "gfr6xh20HF"

def getRequestUrl(url):
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)

    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

def getNaverSearch(node, srcText, start, display):
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % node
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(srcText), start, display)

    url = base + node + parameters
    responseDecode = getRequestUrl(url)

    if (responseDecode == None):
        return None
    else:
        return json.loads(responseDecode)

def getPostData(post, jsonResult, cnt, target_year, target_month):
    pDate = datetime.datetime.strptime(post['pubDate'], '%a, %d %b %Y %H:%M:%S +0900')
    
    # Check if the year and month of the post matches the target year and month
    if pDate.year == target_year and pDate.month == target_month:
        title = post['title']
        description = post['description']
        org_link = post['originallink']
        link = post['link']
        pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')
        jsonResult.append({'cnt': cnt, 'title': title, 'description': description,
                           'org_link': org_link, 'link': org_link, 'pDate': pDate})
        return True
    return False



In [ ]:
# def main():
#     node = 'news'
#     srcText = input('검색어를 입력하세요: ')
#     year = int(input('검색을 원하는 년도를 입력하세요 (예: 2022): '))
#     month = int(input('검색을 원하는 월을 입력하세요 (1-12): '))
#     cnt = 0
#     jsonResult = []

#     jsonResponse = getNaverSearch(node, srcText, 1, 100)
#     total = jsonResponse['total']

#     while ((jsonResponse != None) and (jsonResponse['display'] != 0)):
#         for post in jsonResponse['items']:
#             if getPostData(post, jsonResult, cnt, year, month):
#                 cnt += 1

#         start = jsonResponse['start'] + jsonResponse['display']
#         jsonResponse = getNaverSearch(node, srcText, start, 100)

#     print('전체 검색 : %d 건' % total)

#     with open('%s_naver_%s_%s_%02d.json' % (srcText, node, year, month), 'w', encoding='utf8') as outfile:
#         jsonFile = json.dumps(jsonResult, indent=4, sort_keys=True, ensure_ascii=False)
#         outfile.write(jsonFile)

#     print("가져온 데이터 : %d 건" % (cnt))
#     print('%s_naver_%s_%s_%02d.json SAVED' % (srcText, node, year, month))

In [50]:
def main(srcText, year, month):
    node = 'news'
    cnt = 0
    jsonResult = []
    max_results = 1000  # 최대 결과 개수 설정
    display_count = 100  # 한 번의 요청으로 가져올 수 있는 최대 결과 개수

    start_index = 1
    while start_index <= max_results:
        jsonResponse = getNaverSearch(node, srcText, start_index, display_count)
        total = jsonResponse.get('total', 0)

        if jsonResponse is None or jsonResponse['display'] == 0:
            break

        for post in jsonResponse['items']:
            if getPostData(post, jsonResult, cnt, year, month):
                cnt += 1

        start_index += display_count

    print('전체 검색 : %d 건' % total)

    with open('%s_naver_%s_%s_%02d.json' % (srcText, node, year, month), 'w', encoding='utf8') as outfile:
        jsonFile = json.dumps(jsonResult, indent=4, sort_keys=True, ensure_ascii=False)
        outfile.write(jsonFile)

    print("가져온 데이터 : %d 건" % (cnt))
    print('%s_naver_%s_%s_%02d.json SAVED' % (srcText, node, year, month))


In [51]:
main("시디즈", 2022, 5)

[2023-08-08 16:45:30.332097] Url Request Success
[2023-08-08 16:45:30.450620] Url Request Success
[2023-08-08 16:45:30.540557] Url Request Success
[2023-08-08 16:45:30.691965] Url Request Success
[2023-08-08 16:45:30.852801] Url Request Success
[2023-08-08 16:45:30.945458] Url Request Success
[2023-08-08 16:45:31.043472] Url Request Success
[2023-08-08 16:45:31.153144] Url Request Success
[2023-08-08 16:45:31.235230] Url Request Success
[2023-08-08 16:45:31.333281] Url Request Success
전체 검색 : 8862 건
가져온 데이터 : 105 건
시디즈_naver_news_2022_05.json SAVED


In [19]:
for year in range(2020, 2023):
    for month in range(1, 13):  # 1부터 12까지의 월을 반복
            main("시디즈", year, month)

[2023-08-08 16:20:48.823308] Url Request Success
[2023-08-08 16:20:48.990319] Url Request Success
[2023-08-08 16:20:49.205321] Url Request Success
[2023-08-08 16:20:49.380121] Url Request Success
[2023-08-08 16:20:49.538678] Url Request Success
[2023-08-08 16:20:49.733766] Url Request Success
[2023-08-08 16:20:49.912562] Url Request Success
[2023-08-08 16:20:50.089285] Url Request Success
[2023-08-08 16:20:50.256593] Url Request Success
[2023-08-08 16:20:50.425602] Url Request Success
HTTP Error 400: Bad Request
[2023-08-08 16:20:50.558366] Error for URL : https://openapi.naver.com/v1/search/news.json?query=%EC%8B%9C%EB%94%94%EC%A6%88&start=1001&display=100
전체 검색 : 8862 건
가져온 데이터 : 0 건
시디즈_naver_news_2020_01.json SAVED
[2023-08-08 16:20:50.648869] Url Request Success
[2023-08-08 16:20:50.755017] Url Request Success
[2023-08-08 16:20:50.850313] Url Request Success
[2023-08-08 16:20:50.946163] Url Request Success
[2023-08-08 16:20:51.048716] Url Request Success
[2023-08-08 16:20:51.14879

KeyboardInterrupt: 